In [95]:
import pandas as pd
import urllib.request
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [8]:
sample_filtered = pd.read_csv('sample_filtered.csv')

In [9]:
sample_filtered.columns

Index(['Unnamed: 0', 'id', 'brand', 'imageURLs', 'name', 'prices.amountMax',
       'prices.amountMin', 'prices.color', 'prices.dateSeen', 'prices.size',
       'gender', 'dateDiff', 'avgPrice', 'image_url'],
      dtype='object')

In [10]:
base_dir = os.path.join(os.getcwd())
images_dir = os.path.join(base_dir, 'images')

In [17]:
shoes_tfidf = sample_filtered[['id', 'brand', 'name']]
shoes_tfidf

,id,brand,name
0,AVpe8Pi0ilAPnD_xRYTs,Fireside Casuals,Men's Camouflage Espadrille Slipper
1,AVpfNujoilAPnD_xXXc1,MUK LUKS,Muk Luks Men's Fairisle Full Foot Slipper
2,AVpfEFx41cnluZ0-dGzz,Daxx,Daxx Men's Topstitched Hardware Accent Loafers
3,AVpfOFMeilAPnD_xXe5b,Genuine Dickies,Genuine Dickies Jobrated Maxx Waterproof Work ...
4,AVpfOFMeilAPnD_xXe5b,Genuine Dickies,Genuine Dickies Jobrated Maxx Waterproof Work ...
...,...,...,...
60085,AVpij_dBilAPnD_xFV8S,easy street,Easy Street Natara Women's Sandals
60086,AVpij_dBilAPnD_xFV8S,easy street,Easy Street Natara Women's Sandals
60087,AVpij_dBilAPnD_xFV8S,easy street,Easy Street Natara Women's Sandals
60088,AVpij_dBilAPnD_xFV8S,easy street,Easy Street Natara Women's Sandals


In [18]:
shoes_tfidf['brand'] = shoes_tfidf['brand'].str.lower()
shoes_tfidf['name'] = shoes_tfidf['name'].str.lower()

<ipython-input-18-17d45ebbe204>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shoes_tfidf['brand'] = shoes_tfidf['brand'].str.lower()
<ipython-input-18-17d45ebbe204>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shoes_tfidf['name'] = shoes_tfidf['name'].str.lower()


In [45]:
shoes_tfidf.drop(columns=['bag_of_words'])
shoes_tfidf['bag_of_words'] = shoes_tfidf["brand"] + ' ' +shoes_tfidf["name"]

<ipython-input-45-0c3130afc102>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shoes_tfidf['bag_of_words'] = shoes_tfidf["brand"] + ' ' +shoes_tfidf["name"]


In [47]:
#shoes_tfidf['bag_of_words'] = shoes_tfidf.apply(lambda x : ' '.join(set(x['bag_of_words'].str.split())))

shoes_tfidf['bag_of_words'] = shoes_tfidf.apply(lambda x :' '.join(set(x['bag_of_words'].split())), axis=1)

<ipython-input-47-b1cceef2eabf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shoes_tfidf['bag_of_words'] = shoes_tfidf.apply(lambda x :' '.join(set(x['bag_of_words'].split())), axis=1)


In [61]:
#Taking sample as the cosine matrix taking 26GB space and not able to handle the data
shoes_tfidf_sample = shoes_tfidf[:20000]

In [66]:
shoes_tfidf_sample

,id,brand,name,bag_of_words
0,AVpe8Pi0ilAPnD_xRYTs,fireside casuals,men's camouflage espadrille slipper,camouflage men's fireside slipper espadrille c...
1,AVpfNujoilAPnD_xXXc1,muk luks,muk luks men's fairisle full foot slipper,men's luks fairisle foot muk slipper full
2,AVpfEFx41cnluZ0-dGzz,daxx,daxx men's topstitched hardware accent loafers,accent hardware men's topstitched loafers daxx
3,AVpfOFMeilAPnD_xXe5b,genuine dickies,genuine dickies jobrated maxx waterproof work ...,genuine jobrated work maxx boot waterproof dic...
4,AVpfOFMeilAPnD_xXe5b,genuine dickies,genuine dickies jobrated maxx waterproof work ...,genuine jobrated work maxx boot waterproof dic...
...,...,...,...,...
19995,AV-nbnIEKZqtpbFMSzYa,simply vera vera wang,simply vera vera wang women's all-over stretch...,women's wang stretch simply vera heels all-over
19996,AV-nbnIEKZqtpbFMSzYa,simply vera vera wang,simply vera vera wang women's all-over stretch...,women's wang stretch simply vera heels all-over
19997,AV-nbnIEKZqtpbFMSzYa,simply vera vera wang,simply vera vera wang women's all-over stretch...,women's wang stretch simply vera heels all-over
19998,AV-nbnIEKZqtpbFMSzYa,simply vera vera wang,simply vera vera wang women's all-over stretch...,women's wang stretch simply vera heels all-over


In [62]:
count = CountVectorizer()
count_matrix = count.fit_transform(shoes_tfidf_sample['bag_of_words'])

indices = pd.Series(shoes_tfidf_sample.index)

In [63]:
indices
count_matrix

<20000x1092 sparse matrix of type '<class 'numpy.int64'>'
	with 139960 stored elements in Compressed Sparse Row format>

In [64]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [65]:
cosine_sim.shape

(20000, 20000)

In [72]:
shoes_tfidf_sample[shoes_tfidf_sample['name'] == 'genuine dickies'].index[0]

3

In [78]:
def get_recommendations(name):
    recommended_products = []
    index = shoes_tfidf_sample[shoes_tfidf_sample['name'] == name].index[0]
    
    score_series = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    #print(score_series)
    top_similar_products = list(score_series.iloc[1:5].index)
    
    for i in top_similar_products:
        recommended_products.append(shoes_tfidf_sample['id'][i])
        
    return recommended_products

In [79]:
print(get_recommendations('daxx men\'s topstitched hardware accent loafers'))

['AV-9hpHjYSSHbkXwpyGV', 'AV-9hpHjYSSHbkXwpyGV', 'AV-9hpHjYSSHbkXwpyGV', 'AV-9hpHjYSSHbkXwpyGV']


In [80]:
shoes_tfidf_group = shoes_tfidf.groupby(['id','brand', 'name']).size().reset_index().rename(columns={0:'count'})

In [82]:
shoes_tfidf_group = shoes_tfidf_group.drop(columns=['count'])

In [86]:
shoes_tfidf_group = shoes_tfidf_group.set_index('id')
shoes_tfidf_group['bag_of_words'] = shoes_tfidf_group["brand"] + ' ' +shoes_tfidf_group["name"]
shoes_tfidf_group['bag_of_words'] = shoes_tfidf_group.apply(lambda x :' '.join(set(x['bag_of_words'].split())), axis=1)

In [87]:
count = CountVectorizer()
count_matrix = count.fit_transform(shoes_tfidf_group['bag_of_words'])

indices = pd.Series(shoes_tfidf_group.index)

In [101]:
indices.to_csv('indices.csv')

In [89]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [93]:
cosine_sim

array([[1.        , 0.1767767 , 0.47434165, ..., 0.11785113, 0.23570226,
        0.2236068 ],
       [0.1767767 , 1.        , 0.2236068 , ..., 0.        , 0.16666667,
        0.15811388],
       [0.47434165, 0.2236068 , 1.        , ..., 0.1490712 , 0.2981424 ,
        0.28284271],
       ...,
       [0.11785113, 0.        , 0.1490712 , ..., 1.        , 0.        ,
        0.10540926],
       [0.23570226, 0.16666667, 0.2981424 , ..., 0.        , 1.        ,
        0.21081851],
       [0.2236068 , 0.15811388, 0.28284271, ..., 0.10540926, 0.21081851,
        1.        ]])

In [109]:
indices = pd.read_csv('indices.csv')
def get_recommendations(id):
    recommended_products = []
    index = indices[indices['id'] == id].index[0]
    print(index)
    score_series = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    #print(score_series)
    top_similar_products = list(score_series.iloc[1:5].index)
    
    for i in top_similar_products:
        recommended_products.append(list(shoes_tfidf_group.index)[i])
        
    return recommended_products

In [114]:
print(get_recommendations('AV-9hpHjYSSHbkXwpyGV'))

1
['AV_X3MvzHh53nbDRAY0z', 'AV-nb4rjuC1rwyj_gebo', 'AV-9hsUluC1rwyj_g2eA', 'AV_EdiPRuC1rwyj_hCLM']


In [111]:
indices

,Unnamed: 0,id
0,0,AV-9hndsuC1rwyj_g2bb
1,1,AV-9hpHjYSSHbkXwpyGV
2,2,AV-9hqJYuC1rwyj_g2c_
3,3,AV-9hriAuC1rwyj_g2dv
4,4,AV-9hsUluC1rwyj_g2eA
...,...,...
773,773,AWo0mhdH0U_gzG0he0vq
774,774,AWo0mknt0U_gzG0he0wP
775,775,AWo0mv6FAGTnQPR7rBzB
776,776,AWo0nACN0U_gzG0he0zJ


In [96]:
np.savetxt('cosine_sim.npz', cosine_sim)

In [100]:
vector = np.loadtxt('cosine_sim.npz')
vector

array([[1.        , 0.1767767 , 0.47434165, ..., 0.11785113, 0.23570226,
        0.2236068 ],
       [0.1767767 , 1.        , 0.2236068 , ..., 0.        , 0.16666667,
        0.15811388],
       [0.47434165, 0.2236068 , 1.        , ..., 0.1490712 , 0.2981424 ,
        0.28284271],
       ...,
       [0.11785113, 0.        , 0.1490712 , ..., 1.        , 0.        ,
        0.10540926],
       [0.23570226, 0.16666667, 0.2981424 , ..., 0.        , 1.        ,
        0.21081851],
       [0.2236068 , 0.15811388, 0.28284271, ..., 0.10540926, 0.21081851,
        1.        ]])

In [134]:
shoes_tfidf_group.to_csv('shoes_tfidf_group.csv')

In [141]:
x = pd.read_csv('https://raw.githubusercontent.com/aakash-atnoorkar/Team5_INFO7374_Spring2021/main/Project%3A%20Designer%20Shoes%20Recommendation%20System/Designer%20Shoe%20Recommendation%20App/indices.csv')
#x = x.set_index('id')
x

,id
0,AV-9hndsuC1rwyj_g2bb
1,AV-9hpHjYSSHbkXwpyGV
2,AV-9hqJYuC1rwyj_g2c_
3,AV-9hriAuC1rwyj_g2dv
4,AV-9hsUluC1rwyj_g2eA
...,...
773,AWo0mhdH0U_gzG0he0vq
774,AWo0mknt0U_gzG0he0wP
775,AWo0mv6FAGTnQPR7rBzB
776,AWo0nACN0U_gzG0he0zJ
